In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
  | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
  | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

In [ ]:
!pip install flask-ngrok --quiet
!ngrok config add-authtoken <API_KEY>

In [ ]:
!pip install -U transformers optimum[openvino,nncf] --quiet

from transformers import AutoTokenizer, pipeline
from optimum.intel import OVModelForSequenceClassification

ov_model = OVModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Spam detection/OV/Model")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Spam detection/OV/Tokenizer")
classifier = pipeline("text-classification", model=ov_model, tokenizer=tokenizer)

In [ ]:
!pkill -f ngrok

In [ ]:
import subprocess

#subprocess.Popen(["pkill", "-f", "ngrok"])
subprocess.Popen(["ngrok", "http", "5000"])

from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import request
from flask import jsonify

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/', methods=['POST', 'GET'])
def index():
  return """
  <form action='/classify' method="get">
    <input type="button" value="Add TextBox" onclick="add_field();">
    <input type="button" value="Remove TextBox" onclick="remove_field();">
    <ol id="field_div">

    </ol>
    <button type='submit'>Submit</button>
  </form>

  <script>
    const add_field = () => {
      let total_text = document.getElementsByClassName("input_text").length + 1;
      let field_div = document.getElementById("field_div");
      let new_input = "<li id='input_text_' + total_text><input type='text' class='input_text' name='q'></li>";
      field_div.insertAdjacentHTML('beforeend', new_input);
    }
    const remove_field = () => {
      let total_text = document.getElementsByClassName("input_text").length
      document.getElementById('input_text_' + total_text).remove();
    }
  </script>
  """


@app.route('/classify', methods=['POST', 'GET'])
def classify():
    queries = request.args.getlist('q')
    outputs = classifier(queries)

    results = []

    for i, output in enumerate(outputs):
      label = output['label']
      score = output['score']

      results.append({
        'index': i,
        'text': queries[i],
        'label': label,
        'score': score,
      })


    return jsonify(
      results
    )


app.run()